![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [245]:
import pandas as pd
import numpy as np

df = pd.read_csv('bank_marketing.csv')
df

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,no,cellular,nov,30,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,6,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,41185,56,retired,married,university.degree,no,yes,no,cellular,nov,24,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,41186,44,technician,married,professional.course,no,no,no,cellular,nov,17,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [246]:
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
client.rename(columns={'client_id':'id'}, inplace = True)

campaign = df[['client_id', 'campaign', 'month', 'day', 'duration', 'pdays', 'previous', 'poutcome', 'y']]
campaign.rename(columns={'duration':'contact_duration', 'previous':'previous_campaign_contacts', 'y':'campaign_outcome', 'poutcome':'previous_outcome', 'campaign':'number_contacts'}, inplace = True)

economics = df[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]
economics.rename(columns={'euribor3m':'euribor_three_months', 'nr_employed':'number_employed'}, inplace = True)

In [247]:
campaign

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,999,0,nonexistent,no
1,1,1,may,19,149,999,0,nonexistent,no
2,2,1,may,23,226,999,0,nonexistent,no
3,3,1,may,27,151,999,0,nonexistent,no
4,4,1,may,3,307,999,0,nonexistent,no
...,...,...,...,...,...,...,...,...,...
41183,41183,1,nov,30,334,999,0,nonexistent,yes
41184,41184,1,nov,6,383,999,0,nonexistent,no
41185,41185,2,nov,24,189,999,0,nonexistent,no
41186,41186,1,nov,17,442,999,0,nonexistent,yes


In [248]:
client['education'].replace(".", '_', inplace=True)
client['education'].replace("unknown", np.nan, inplace=True)
client

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,no
41184,41184,46,blue-collar,married,professional.course,no,no,no
41185,41185,56,retired,married,university.degree,no,yes,no
41186,41186,44,technician,married,professional.course,no,no,no


In [249]:
client['job'] = [i.replace('.', '') for i in client['job']]
client

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,no
41184,41184,46,blue-collar,married,professional.course,no,no,no
41185,41185,56,retired,married,university.degree,no,yes,no
41186,41186,44,technician,married,professional.course,no,no,no


In [250]:
campaign['previous_outcome'] = campaign['previous_outcome'].replace('success', 1).replace('failure', 0).replace('nonexistent', np.nan)
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace('yes', 1).replace('no', 0).replace('nonexistent', np.nan)
campaign

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,999,0,NaN,0
1,1,1,may,19,149,999,0,NaN,0
2,2,1,may,23,226,999,0,NaN,0
3,3,1,may,27,151,999,0,NaN,0
4,4,1,may,3,307,999,0,NaN,0
...,...,...,...,...,...,...,...,...,...
41183,41183,1,nov,30,334,999,0,NaN,1
41184,41184,1,nov,6,383,999,0,NaN,0
41185,41185,2,nov,24,189,999,0,NaN,0
41186,41186,1,nov,17,442,999,0,NaN,1


In [251]:
campaign['campaign_id'] = 1

In [252]:
campaign['last_contact_date'] = pd.to_datetime('2022' + '-' + df['month'] + '-' + df['day'].astype(str))

In [253]:
economics.columns

Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')

In [254]:
campaign.drop(['month', 'day', ], axis=1)

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,NaN,0,1,2022-05-13
1,1,1,149,999,0,NaN,0,1,2022-05-19
2,2,1,226,999,0,NaN,0,1,2022-05-23
3,3,1,151,999,0,NaN,0,1,2022-05-27
4,4,1,307,999,0,NaN,0,1,2022-05-03
...,...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,NaN,1,1,2022-11-30
41184,41184,1,383,999,0,NaN,0,1,2022-11-06
41185,41185,2,189,999,0,NaN,0,1,2022-11-24
41186,41186,1,442,999,0,NaN,1,1,2022-11-17


In [255]:
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

In [256]:
client_table = """
CREATE TABLE client (
id SERIAL PRIMARY KEY,
age INT,
job TEXT,
marital TEXT,
education TEXT,
credit_default BOOL,
housing BOOL,
loan BOOL
)

\copy client from 'client.csv' DELIMITER ',' CSV HEADER

"""

In [257]:
campaign_table = """
CREATE TABLE campaign 
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client (id)
    number_contacts INT,
    contact_duration INT,
    pdays INT,
    previous_campaign_contacts INT,
    previous_outcome BOOL,
    campaign_outcome BOOL,
    last_contact_date DATE
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

In [258]:
# campaign_table = """CREATE TABLE campaign
# (
#     campaign_id SERIAL PRIMARY KEY,
#     client_id SERIAL references client (id),
#     number_contacts INTEGER,
#     contact_duration INTEGER,
#     pdays INTEGER,
#     previous_campaign_contacts INTEGER,
#     previous_outcome BOOLEAN,
#     campaign_outcome BOOLEAN,
#     last_contact_date DATE    
# );
# \copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
# """

In [259]:
economics_table = """
CREATE TABLE economics 
(
    client_id SERIAL REFERENCES client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
)
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""